In [1]:
import pandas as pd
import numpy as np
import os
import re
import random

In [2]:
from datetime import timedelta

In [3]:
import scipy.stats as sp

In [4]:
def PrepareData(df,wiki_measure,names_trade):
    beg_wiki = df.groupby('name').apply(lambda a: a[~a[wiki_measure].isna()]['time'].min()).to_frame()
    beg_price = df.groupby('name').apply(lambda a: a[~a['price'].isna()]['time'].min()).to_frame()
    beg_all = pd.merge(beg_wiki,beg_price,left_index=True,right_index=True,suffixes=['_wiki','_price'])
    beg = beg_all.max(axis=1)
    df = df[df['name'].isin(beg.index)].groupby('name',as_index=False).apply(lambda a: a[a['time'] >= beg[a['name'].iloc[0]]])
    
    
    df = df.drop_duplicates()
    df = df[df['volume'] >= 0]
    
    df.sort_values('time',inplace=True)
    df = df.set_index('time',drop=False)
    
    traded_db =df[(df['name'].isin(names_trade))]
    traded_db = traded_db.groupby('name',as_index=False).apply(lambda a: a[a['time'] >= (a[a['volume'] > 100000].min()['time'])])
    
    
    traded_db = traded_db.groupby('name').apply(lambda a: a.resample('D',on='time').mean()).reset_index()
    
    return traded_db

In [5]:
names_trade = ['Ethereum', 'Ripple', 'Litecoin', 'Bitcoin Cash',  'Monero','Tether'
                ,'Dash',  'Stellar','Dogecoin','EOS','Ethereum Classic', 'NEO', 'OmiseGO',
                   'Bitcoin Gold','Cardano','Zcash','Bitcoin']

In [6]:
db_wiki = pd.read_csv('Data/wiki_market.csv',index_col=0,dtype={'sym':str},parse_dates=['time']) #pd.read_csv('../Data/wiki_all_df_all_lang.csv',index_col=0)

In [7]:
traded_db = PrepareData(db_wiki,'views_page',names_trade)

In [8]:
traded_db = traded_db.groupby('name',as_index=False).apply(lambda a: a.sort_values('time')).set_index('time',drop=False)

In [9]:
traded_db.index.names = ['Date']

In [10]:
traded_db_delta = pd.merge(traded_db.groupby('name').apply(lambda a: a['views_page'].diff().dropna()).to_frame('views_page_delta').reset_index(),
        traded_db,left_on = ['name','Date'],right_on = ['name','time'],how ='outer').set_index('Date')

In [11]:
traded_db_delta = pd.merge(traded_db.groupby('name').apply(lambda a: a['price'].diff().dropna()).to_frame('price_delta').reset_index(),
        traded_db_delta,left_on = ['name','Date'],right_on = ['name','time'],how='outer').set_index('Date')

In [12]:
traded_db_delta = traded_db_delta[['name','price','views_page','price_delta','views_page_delta','time']]

In [13]:
traded_db_delta = traded_db_delta.groupby('name').apply(lambda a: a.resample('D',on='time').mean()).reset_index()

In [14]:
def BuySellStrategy(df_delta,social_name,save_name,t_delta,folder):
    if social_name == 'price reversed':
        positives = df_delta[df_delta['price_delta'] <= 0] 
        negatives = df_delta[df_delta['price_delta'] > 0]
    else:
        positives = df_delta[df_delta['%s_delta'%social_name] > 0] 
        negatives = df_delta[df_delta['%s_delta'%social_name] <= 0]
    
    positives = positives.assign(time_1 = lambda x: (x['time'] + timedelta(days =t_delta)))
    positives = positives.assign(time_2 = lambda x: (x['time_1'] + timedelta(days =t_delta)))
    
    positive_merged = pd.merge(pd.merge(positives[['price','name','time','time_1','time_2']],df_delta[['time','name','price']],
        left_on=['time_1','name'],right_on=['time','name'],suffixes=['_before','_now']),
        df_delta[['time','name','price']],left_on=['time_2','name'],right_on=['time','name'],how='left')
    positive_merged.set_index('time_1',inplace =True,drop=False)
    if positive_merged.count()['time'] > 0:
        positive_merged = positive_merged[~positive_merged['price_now'].isna()]
       
    
    returns_postives = (np.log(positive_merged['price_now']) - np.log(positive_merged['price'])).to_frame('returns').reset_index()
    
    returns_postives = returns_postives.rename(columns = {'time_1':'time'})
    
    
    negatives =negatives.assign(time_1 = lambda x:  (x['time'] + timedelta(days =t_delta)))
    negatives = negatives.assign(time_2 = lambda x: (x['time_1'] + timedelta(days =t_delta)))
    negatives_merged = pd.merge(pd.merge(negatives[['price','name','time','time_1','time_2']],df_delta[['time','name','price']],
        left_on=['time_1','name'],right_on=['time','name'],suffixes=['_before','_now']),
        df_delta[['time','name','price']],left_on=['time_2','name'],right_on=['time','name'],how='left')
    negatives_merged.set_index('time_2',inplace =True,drop=False)
    if negatives_merged.count()['time'] > 0:
        negatives_merged = negatives_merged[~negatives_merged['price_now'].isna()]
       
    returns_negatives = (np.log(negatives_merged['price']) - np.log(negatives_merged['price_now'])).to_frame('returns').reset_index()
    returns_negatives = returns_negatives.rename(columns = {'time_2':'time'})
    returns_df = ((pd.merge(returns_negatives,returns_postives,on='time',how='outer')).set_index('time').sum(axis=1)).to_frame('returns').reset_index().groupby('time').mean().reset_index()
    returns_df.fillna(0,inplace=True)
    
    returns_df.to_csv('%s/returns_%s.csv'%(folder,save_name))
    return returns_df,negatives_merged,positive_merged
        

In [15]:
def GetOneRand(df):
    dic_ret = {}
    i = 0
    for grp_name,grp in df.groupby('name'):
        #print(grp_name)
        dates = grp[grp['time'] < grp['time'].max()].resample('2D',on='time').median().index
        for date_s in dates: 
            date = date_s.date()
            i+=1
            rand = np.random.rand()
            if rand <= 0.5:
                ret = (np.log(grp.loc[date]['price'])-np.log(grp.loc[date+timedelta(1)]['price']))
            else:
                ret =(np.log(grp.loc[date+timedelta(1)]['price'])-np.log(grp.loc[date]['price']))
            dic_ret[i] = {'name':grp_name,'time':date,'return':ret} 
    return pd.DataFrame.from_dict(dic_ret,orient='index')

In [16]:
def GetAllRand(df,n_rand,start,folder_name):
    returns_df = GetOneRand(df)
    returns_df['time'] = pd.to_datetime(returns_df['time'])
    returns_df = returns_df.assign(run= lambda a: 0 )
    df_ls = [returns_df]
    n = 0
    while n < n_rand:
        first_run = GetOneRand(df)
        first_run['time'] = pd.to_datetime(first_run['time'])
        first_run = first_run.assign(run= lambda a: n)
        df_ls.append(first_run)
        
        n += 1
    
    pd.concat(df_ls).to_csv('%s/%s_%s.csv'%(folder_name,
                        start,'Rand_views_page_updated'))

# The Calling!

In [17]:
folder_name = 'Data'

In [18]:
df_wbs,dn,dp = BuySellStrategy(traded_db_delta,'views_page','wiki',1,folder_name)



In [19]:
df_vbs,dn_p,dp_p = BuySellStrategy(traded_db_delta,'price','price',1,folder_name)

In [20]:
def GetReturnsName(df,name,pos_neg_flag):
    df.set_index('time_1',inplace =True,drop=False)
    if df.count()['time'] > 0:
        df = df[~df['price_now'].isna()]
    
    if pos_neg_flag == 'positive':
        ret_df = (np.log(df['price_now']) - np.log(df['price'])).to_frame('returns').reset_index()
    
        ret_df = ret_df.rename(columns = {'time_1':'time'})
    else:
        
       
        ret_df = (np.log(df['price']) - np.log(df['price_now'])).to_frame('returns').reset_index()
        ret_df = ret_df.rename(columns = {'time_2':'time'})
    return ret_df
    
    

In [21]:
suzi_returns_per_crypt  = pd.merge(dp.groupby('name').apply(lambda a : GetReturnsName(a,a['name'].iloc[0],'positive')).reset_index(),
          dn.groupby('name').apply(lambda a : GetReturnsName(a,a['name'].iloc[0],'negative')).reset_index(),left_on=['name','level_1'],
        right_on=['name','level_1'],how='outer').set_index(['name','level_1']).sum(axis=1).reset_index()#.groupby('name').mean()

In [22]:
price_returns_per_crypt = pd.merge(dp_p.groupby('name').apply(lambda a : GetReturnsName(a,a['name'].iloc[0],'positive')).reset_index(),
          dn_p.groupby('name').apply(lambda a : GetReturnsName(a,a['name'].iloc[0],'negative')).reset_index(),left_on=['name','level_1'],
        right_on=['name','level_1'],how='outer').set_index(['name','level_1']).sum(axis=1).reset_index()#.groupby('name').mean()

In [23]:
wiki_returns_per_crypt.rename(columns={0:'wikipedia'},inplace=True)

In [24]:
price_returns_per_crypt.rename(columns={0:'price'},inplace=True)

In [26]:
GetAllRand(traded_db,1000,'',folder_name)


In [26]:
#rands  = pd.read_csv('Data/_Rand_views_page_updated.csv',index_col=0,parse_dates=['time'])

In [38]:
views_st_returns = pd.read_csv('Data/returns_wiki.csv',parse_dates=['time'],index_col=0)
price_st_returns = pd.read_csv('Data/returns_price.csv',parse_dates=['time'],index_col=0)


In [39]:
rand_daily_returns = rands.groupby('run').apply(lambda a: a.groupby('time').sum()['return']).mean(axis=0).to_frame('return')

In [40]:
rand_crypt = rands.groupby('name').apply(lambda a: a.groupby('run').sum().mean())

In [41]:
rand_daily_returns.to_csv('Data/returns_random.csv')

In [49]:
pd.merge(pd.merge(wiki_returns_per_crypt.groupby('name').sum()['wikipedia'].to_frame(),
        price_returns_per_crypt.groupby('name').sum()['price'].to_frame(),left_index=True,
        right_index=True).rename(columns={'0_x':'wikipedia','0_y':'price'}),rand_crypt,left_index=True,
         right_index=True).rename(columns={'return':'rand'}).to_csv(
    'Data/returns_per_crypt.csv')

In [44]:
returns_df_daily = pd.merge(pd.merge(views_st_returns.set_index('time')['returns'].to_frame('wikipedia'),
     price_st_returns.set_index('time')['returns'].to_frame('price'),left_index=True,right_index=True,how='outer'),
        (rand_daily_returns['return']).to_frame('rand'),left_index=True,right_index=True,how='outer')

In [45]:
returns_df_daily = returns_df_daily[returns_df_daily.index>'2015-07-01']

In [46]:
dates_ava = pd.date_range('2015-07-01',returns_df_daily.index.max(),freq='2M',closed='left')

In [47]:
st_ends_ls = []
i = 0
for date_now in dates_ava:
    i +=1
    for date_end in dates_ava:
        rolling_df = returns_df_daily[(returns_df_daily.index >= date_now)&
                                      (returns_df_daily.index <= date_end)]
        res = rolling_df.sum().to_frame().transpose().assign(start=lambda a: date_now)
        res = res.assign(end=lambda a: date_end)
        st_ends_ls.append(res)
returns_start_ends = pd.concat(st_ends_ls)
returns_start_ends = returns_start_ends[returns_start_ends['start']!= returns_start_ends['end']]
returns_start_ends = returns_start_ends[returns_start_ends['end'] != '2015-07-01']
returns_start_ends = returns_start_ends[returns_start_ends['start'] != '2019-01-01']

In [48]:
returns_start_ends.to_csv('Data/start_ends.csv')